In [ ]:
#@title Installation of OpenAI API and langchain
#@markdown Note: Make sure to run all code cells (play button next to code cells or Runtime->Run all)
%pip install langchain==0.0.140
%pip install openai

In [5]:
#@title Insert API key and scenario

openai_api_key = 'sk-abcdef123... !Only GPT-4 access-enabled API keys are supported!' #@param {type:"string"}

#@markdown Instructions: 
#@markdown - You'll need an OpenAI API key with access to GPT-4!
#@markdown - Provide a starting point for the simulation. Optional: Can include/imply a perturbation
#@markdown   - e.g., a situation or experimental setup or a detailed/complex question that will be answered using a simulation.
#@markdown   - for more examples, check out the `prompts/` folders [in our repository](https://github.com/OpenBioLink/SimulateGPT/tree/main/experiments)
#@markdown - If you expect a final outcome, explicitly request it (use the words) ‘final outcome’
#@markdown - Optional: You can increase the novelty by adding: "Focus on more novelty."
#@markdown - SimulateGPT can be used to ask detailed/complex questions about biology. 
#@markdown - SimulateGPT has the potential to assess the question in more depth and provide more informed answers than the default ChatGPT.
#@markdown - For chat-like follow-up questions, refer to the OpenAI playground and provide the SimulateGPT prompt as SYSTEM message

simulation_scenario = 'A wild type mouse is injected with YUMM 1.7. Report the most relevant final outcome.' #@param {type:"string"}

#@markdown Use the default SimulateGPT prompt or one of variations, described in the paper.

complexity_input = "SimulateGPT (default)" #@param ["SimulateGPT (default)", "Low-complexity SimulateGPT", "5-step SimulateGPT"]

# translate to prompt filename
complexity_fn = dict(zip(
    ["SimulateGPT (default)", "Low-complexity SimulateGPT", "5-step SimulateGPT"], 
    ["high_complexity", "low_complexity", "high5step_complexity"]))[complexity_input]


In [6]:
#@title Load OpenAI API and SimulateGPT

import os 
import urllib.request
from langchain.callbacks.base import CallbackManager

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

def load_openai_api(api_key: str, model="gpt-4"):
    """
    Args:
        api_key: The OpenAI API key to use. If None, then try to load it from the environment variable OPENAI_API_KEY or from the password store.
        model: The model to use. Defaults to "gpt-4". alternatively use gpt-3.5-turbo for faster and cheaper exploration
    """

    os.environ["OPENAI_API_KEY"] = api_key
    stream_chat = ChatOpenAI(
        streaming=True,
        temperature=0,
        model_name=model,
        request_timeout=60,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        verbose=True,
    )
    return stream_chat
stream_chat = load_openai_api(openai_api_key)

# Load the system prompt from GitHub
simulate_gpt_url = f"https://raw.githubusercontent.com/OpenBioLink/SimulateGPT/main/system_messages/{complexity_fn}"

response = urllib.request.urlopen(simulate_gpt_url)
data = response.read()
simulate_gpt_system = data.decode("utf-8")
print("Loaded API and SimulateGPT")

Loaded API and SimulateGPT


In [7]:
#@title Run SimulateGPT through OpenAI GPT-4 API

request = [
    SystemMessage(content=simulate_gpt_system),
    HumanMessage(content=simulation_scenario),
]
result = stream_chat(request)

parameters:
  - wild type mouse
  - YUMM 1.7 injection

simulation:
  - step: 1
    level: cellular
    facts: YUMM 1.7 is a murine melanoma cell line derived from a BRAFV600E-driven melanoma in a C57BL/6 mouse [1]. Melanoma is a type of skin cancer that arises from melanocytes, the pigment-producing cells [2]. The BRAFV600E mutation is a common driver of melanoma, leading to constitutive activation of the MAPK/ERK signaling pathway, promoting cell proliferation and survival [3].
    entities: YUMM 1.7, melanoma, melanocytes, BRAFV600E, MAPK/ERK signaling pathway
    assumptions: The injected YUMM 1.7 cells are viable and capable of proliferating in the wild type mouse.
    consequence: engraftment of YUMM 1.7 cells in the mouse
    probability: 80
    explanation: The YUMM 1.7 cells are derived from a C57BL/6 mouse, which is the same genetic background as the wild type mouse. This increases the likelihood of successful engraftment and growth of the melanoma cells in the mouse.
    nov